# 02.A: Working with Datasets

It's important to have a clear and sensible way of representing the datasets that learning algorithms train on. A dataset consists of $n$ examples. Each example consists of $m$ features. This makes $m$ the number dimensions the dataset has. In supervised learning, the dataset is a matrix like this:

$\boldsymbol{D} =\left[\begin{array}{cccccc} 
  x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_m^{(1)} & y^{(1)}\\ 
  x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_m^{(2)} & y^{(2)}\\
  x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_m^{(3)} & y^{(3)}\\
  \vdots    & \vdots    & \vdots    & \cdots & \vdots & \vdots \\
  x_1^{(n)} & x_2^{(n)} & x_3^{(n)} & \cdots & x_m^{(n)} & y^{(n)}
\end{array}\right]$

Each row of this matrix is an example consisting of the $m$ features plus the target label as the last element in the row. In other words, $\boldsymbol{D}$ consists of both the input matrix $\boldsymbol{X}$ and target vector $y$, where: 

$\boldsymbol{X} =\left[\begin{array}{ccccc} 
  x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_m^{(1)}\\ 
  x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_m^{(2)}\\
  x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_m^{(3)}\\
  \vdots    & \vdots    & \vdots    & \cdots & \vdots \\
  x_1^{(n)} & x_2^{(n)} & x_3^{(n)} & \cdots & x_m^{(n)}
\end{array}\right]$

and

$\boldsymbol{y} =\left[\begin{array}{c} 
  y^{(1)}\\ 
  y^{(2)}\\
  y^{(3)}\\
  \vdots \\
  y^{(n)}
\end{array}\right]$

For unsupervised learning, $\boldsymbol{D}$ is the same as $\boldsymbol{X}$. Here is a class named `DataSet` to represent datasets. It uses pandas' DataFrame.

Another name for $X$ is `inputs`, and another name for $y$ is `target`. In addition, features have names. Let's put all of this together in a class that we will be using in subsequent weeks.

In [1]:
import numpy as np
import pandas as pd

#testing this repository

class DataSet:
    """
    A dataset for a machine learning problem. A dataset d has the following properties:
    d.examples   A list of examples. Each one contains both the features and the target.
    d.features   An array of the of feature names.
    d.target     An m by 1 array containing the values of y
    d.y          Same as d.target
    d.inputs     An n by m array containing the values of X
    d.X          Same as d.inputs
    d.N          Number of examples
    d.M          Number of dimensions
    d.name       The name of the data set (for output display only)
    
    """
    def __init__(self, data, features=None, y=None, name=None):
        """
        If y is True, the data contains the target as the last column
        If y is None or False, No target is available
        Else y is an array to be added as the last column of the examples  dataframe
        """
        self.__name = name
        if isinstance(data, pd.DataFrame):
            self.__examples = data
        else:
            self.__examples = pd.DataFrame(data, columns=features)
            
        if y is True:
            self.__examples.columns = [*self.__examples.columns[:-1], 'y']
        elif y is not False and y is not None:
            self.__examples['y'] = y
    
    @property
    def examples(self):
        return self.__examples
    
    @property
    def features(self):
        return self.__examples.columns[:-1].values
    
    @property
    def target(self):
        if 'y' in self.__examples.columns:
            return self.__examples['y'].values.reshape(self.N, 1)
        return None
    
    @property
    def y(self):
        return self.target
    
    @property
    def inputs(self):
        return self.__examples.iloc[:, :-1].values
    
    @property
    def X(self):
        return self.inputs
    
    @property
    def name(self):
        return self.__name
    
    @property
    def N(self):
        return self.__examples.shape[0]
    
    @property
    def M(self):
        return self.inputs.shape[1]
    
    def shuffled(self, random_state=None):
        rgen = np.random.RandomState(random_state)
        indexes = np.arange(self.N)
        rgen.shuffle(indexes)
        return DataSet(self.__examples.iloc[indexes])
    
    def train_test_split(self, start=0, end=None, test_portion=None, shuffle=False, random_state=None):
        """
        Splits the dataset into a training set and atest set. 
        If test_portion is specified, return that portion of the dataset as test 
        and the rest as training. 
        Otherwise, return the examples between start and end as test and the 
        rest as training.
        """
        indexes = np.arange(self.N)
        if shuffle is True:
            rgen = np.random.RandomState(random_state)
            rgen.shuffle(indexes)

        if test_portion is None:
            end = end or self.N
        else:
            if not isinstance(test_portion, float) or test_portion < 0 or test_portion > 1:
                raise TypeError("Only fractions between ]0,1[ are allowed")

            start = self.N - int(self.N * test_portion)
            end = self.N

        test = DataSet(self.examples.iloc[indexes[range(start, end)]])
        train = DataSet(pd.concat([self.examples.iloc[indexes[range(start)]], 
                                      self.examples.iloc[indexes[range(end, self.N)]]], axis=0))    
        return train, test
    
    def __repr__(self):
        return repr(self.examples)

This class has a couple of properties including `name` (informational), `features` (the names of the features), `inputs`, `target`, `X`, `y`, `N` (number of examples), `M` (number of dimensions).

A DataSet object is created using a NumPy array or a Pandas dataframe. If it is a NumPy array, the class uses it to create a Pandas dataframe. The dataframe storing the data can be retrieved back using the `examples` property.


Let's test this class by creating a $27 \times 3$ input data and a separate $y$ column.

In [2]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")

ds

     x1   x2         x1  y
0   8.0  7.0   8.297619  0
1   3.0  5.0   7.937979  0
2   6.0  7.0  10.038736  0
3   7.0  6.0  12.183313  0
4   6.0  5.0   9.677187  1
5   7.0  5.0  13.299628  0
6   3.0  5.0   6.916411  0
7   5.0  2.0   8.316049  1
8   5.0  8.0   8.174413  0
9   2.0  6.0   8.533317  1
10  5.0  2.0   7.633054  0
11  5.0  2.0  13.573022  1
12  3.0  7.0   9.765295  1
13  4.0  6.0  10.032035  0
14  3.0  4.0  11.214626  1
15  8.0  5.0   8.220673  1
16  6.0  1.0   8.677240  0
17  7.0  7.0  13.794173  1
18  2.0  3.0  12.115598  1
19  5.0  8.0   6.226230  1
20  4.0  4.0   8.891929  0
21  2.0  5.0   5.018717  0
22  3.0  6.0  12.738946  0
23  4.0  1.0   9.694300  0
24  2.0  3.0   8.991872  0
25  8.0  4.0  11.070437  1
26  5.0  4.0  10.459243  1

In [3]:
ds.examples

,x1,x2,x1,y
0,8.0,7.0,8.297619,0
1,3.0,5.0,7.937979,0
2,6.0,7.0,10.038736,0
3,7.0,6.0,12.183313,0
4,6.0,5.0,9.677187,1
5,7.0,5.0,13.299628,0
6,3.0,5.0,6.916411,0
7,5.0,2.0,8.316049,1
8,5.0,8.0,8.174413,0
9,2.0,6.0,8.533317,1


In [4]:
ds.features

array(['x1', 'x2', 'x1'], dtype=object)

In [5]:
ds.target 

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1]])

In [6]:
ds.y 

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1]])

In [7]:
ds.inputs

array([[ 8.        ,  7.        ,  8.29761949],
       [ 3.        ,  5.        ,  7.93797935],
       [ 6.        ,  7.        , 10.038736  ],
       [ 7.        ,  6.        , 12.18331277],
       [ 6.        ,  5.        ,  9.67718737],
       [ 7.        ,  5.        , 13.29962828],
       [ 3.        ,  5.        ,  6.91641086],
       [ 5.        ,  2.        ,  8.31604889],
       [ 5.        ,  8.        ,  8.17441264],
       [ 2.        ,  6.        ,  8.53331747],
       [ 5.        ,  2.        ,  7.63305378],
       [ 5.        ,  2.        , 13.57302212],
       [ 3.        ,  7.        ,  9.76529509],
       [ 4.        ,  6.        , 10.03203488],
       [ 3.        ,  4.        , 11.21462631],
       [ 8.        ,  5.        ,  8.22067304],
       [ 6.        ,  1.        ,  8.67724021],
       [ 7.        ,  7.        , 13.79417288],
       [ 2.        ,  3.        , 12.11559806],
       [ 5.        ,  8.        ,  6.2262297 ],
       [ 4.        ,  4.        ,  8.891

In [8]:
ds.X

array([[ 8.        ,  7.        ,  8.29761949],
       [ 3.        ,  5.        ,  7.93797935],
       [ 6.        ,  7.        , 10.038736  ],
       [ 7.        ,  6.        , 12.18331277],
       [ 6.        ,  5.        ,  9.67718737],
       [ 7.        ,  5.        , 13.29962828],
       [ 3.        ,  5.        ,  6.91641086],
       [ 5.        ,  2.        ,  8.31604889],
       [ 5.        ,  8.        ,  8.17441264],
       [ 2.        ,  6.        ,  8.53331747],
       [ 5.        ,  2.        ,  7.63305378],
       [ 5.        ,  2.        , 13.57302212],
       [ 3.        ,  7.        ,  9.76529509],
       [ 4.        ,  6.        , 10.03203488],
       [ 3.        ,  4.        , 11.21462631],
       [ 8.        ,  5.        ,  8.22067304],
       [ 6.        ,  1.        ,  8.67724021],
       [ 7.        ,  7.        , 13.79417288],
       [ 2.        ,  3.        , 12.11559806],
       [ 5.        ,  8.        ,  6.2262297 ],
       [ 4.        ,  4.        ,  8.891

In [9]:
ds.name

'Sample Data'

In [10]:
ds.N

27

In [11]:
ds.M

3

## Shuffling

The above class also supports a few useful methods. One such method is for shuffling the data, which we do often before training. This method returns a new DataSet instance with the shuffled data. Here is how this method is implemented:

```python
    ...
    def shuffled(self, random_state=None):
        rgen = np.random.RandomState(random_state)
        indexes = np.arange(self.N)
        rgen.shuffle(indexes)
        return DataSet(self.__examples.iloc[indexes])
   ...
```

Here is an example using this function.

In [12]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ds.shuffled()

     x1   x2         x1  y
1   7.0  4.0  11.681516  1
18  8.0  6.0  11.676361  1
7   7.0  6.0  11.125015  0
26  8.0  5.0   7.473321  0
20  8.0  1.0  12.522537  1
15  3.0  7.0  10.705230  0
11  2.0  1.0   9.044078  1
17  4.0  2.0  12.994948  1
19  5.0  2.0  11.811410  1
22  6.0  4.0  13.902294  1
2   8.0  8.0  10.383599  0
14  2.0  5.0  12.767474  0
5   2.0  3.0   9.265486  0
8   6.0  4.0   9.321202  0
16  3.0  3.0  11.519305  1
12  6.0  7.0   9.051530  1
13  6.0  7.0  10.273107  1
21  7.0  1.0   8.798935  1
24  5.0  4.0   8.960831  0
6   6.0  8.0   8.359678  0
0   7.0  8.0   8.933853  0
23  2.0  7.0  11.100170  0
4   2.0  8.0   8.675031  1
3   4.0  2.0   9.443225  1
10  2.0  4.0  11.782023  1
9   6.0  8.0   9.104118  0
25  2.0  4.0  11.141692  0

## Splitting a dataset into training and test datasets

Another useful method provided by the above dataset class is the `train_test_split` method. This method splits the dataset into a training and test sets. Here is how this method is implemented:

```python
    ...
    def train_test_split(self,start=0, end=None, test_portion=None, shuffle=False, random_state=None):
        """
        Splits the dataset into a training set and atest set. 
        If test_portion is specified, return that portion of the dataset as test 
        and the rest as training. 
        Otherwise, return the examples between start and end as test and the 
        rest as training.
        """
        indexes = np.arange(self.N)
        if shuffle is True:
            rgen = np.random.RandomState(random_state)
            rgen.shuffle(indexes)

        if test_portion is None:
            end = end or self.N
        else:
            if not isinstance(test_portion, float) or test_portion < 0 or test_portion > 1:
                raise TypeError("Only fractions between ]0,1[ are allowed")

            start = self.N - int(self.N * test_portion)
            end = self.N

        test = DataSet(self.examples.iloc[indexes[range(start, end)]])
        train = DataSet(pd.concat([self.examples.iloc[indexes[range(start)]], 
                                      self.examples.iloc[indexes[range(end, self.N)]]], axis=0))    
        return train, test
   ...
```

If the `start` and end `end` parameters exist, the method returns the examples before them as test and the rest of the data as training. If `test_portion` is provided, then that portion of the data is returned as test and the rest as training. The `shuffle` parameter can be used to instruct the method to shuffle the data before splitting it. The method finally returns two dataset instances: training and test sets.

Here is an example using this method.

In [13]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, te = ds.train_test_split(test_portion=.25, shuffle=False, random_state=17)
print('Training set = \n', ta)
print('Test set = \n', te)

Training set = 
      x1   x2         x1  y
0   8.0  6.0   6.379799  1
1   3.0  1.0  10.590870  0
2   7.0  5.0   7.918732  1
3   3.0  7.0  12.414473  0
4   5.0  8.0   7.277684  0
5   4.0  5.0  10.071677  0
6   8.0  2.0   8.512784  1
7   7.0  6.0  11.521260  0
8   2.0  8.0   7.431489  0
9   3.0  7.0  13.411027  1
10  4.0  4.0  13.670948  1
11  6.0  5.0   7.185314  1
12  4.0  2.0  12.590617  1
13  4.0  2.0   7.258792  0
14  4.0  5.0   9.115850  0
15  8.0  2.0   9.533879  0
16  6.0  1.0   8.634883  0
17  4.0  6.0   9.237838  0
18  6.0  8.0  10.155867  0
19  5.0  6.0  12.325432  1
20  6.0  6.0  13.186454  1
Test set = 
      x1   x2         x1  y
21  6.0  8.0   8.921916  1
22  8.0  3.0  11.266343  0
23  2.0  7.0  11.571966  1
24  7.0  6.0  11.546789  1
25  7.0  6.0   8.381000  0
26  3.0  7.0   8.933807  0


## Using this dataset class inside other notebooks

This class is part of the `mylib` library of this class with is provided to you. Here is how to import this library:

In [14]:
import mylib as my

Once imported, one can use it like this:

In [15]:
ds = my.DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, te = ds.train_test_split(test_portion=.25, shuffle=False, random_state=17)
print('Training set = \n', ta)
print('Test set = \n', te)

Training set = 
      x1   x2         x1  y
0   7.0  3.0  12.715125  0
1   8.0  2.0  13.175442  0
2   7.0  2.0  11.787256  0
3   3.0  6.0  10.999183  1
4   8.0  7.0  10.434643  0
5   5.0  5.0  12.617252  0
6   4.0  2.0  12.421318  0
7   7.0  5.0   7.000473  0
8   2.0  4.0   7.808557  0
9   6.0  2.0   6.923458  0
10  6.0  8.0   8.579182  0
11  3.0  5.0   9.922621  1
12  3.0  4.0  11.188480  1
13  3.0  2.0  10.947124  0
14  7.0  5.0  16.118540  1
15  6.0  1.0  11.699667  0
16  5.0  7.0  10.982453  1
17  3.0  3.0  11.804204  1
18  2.0  2.0  10.631997  1
19  5.0  2.0   8.827421  1
20  8.0  5.0  11.392963  0
Test set = 
      x1   x2         x1  y
21  8.0  4.0   8.505845  1
22  2.0  2.0   9.927662  1
23  3.0  7.0  12.016942  0
24  8.0  6.0   7.573557  0
25  2.0  4.0   9.686708  0
26  6.0  3.0   8.843435  1


## EXERCISE

Refactor the above DataSet class by adding a method named `train_validation_test_split` to it. This method should split the data into three sets: training, validation, and test. This method should receive a dictionary parameter named `portions` specifying how much of the data is in each set. For a 75%/15%/10% split, one can use the following portions parameter:

```python
portions={"training": .75, 'validation': .15, 'test': .10 }
```

The method should support the `shuffle` parameter as well. You may call the `train_test_split` method internally. Make sure to include a comment describing how your implementation of the method works. Test your method on the `ds` dataset above and show that it works.

In [16]:
# TODO